In [1]:
# u.user 파일을 DataFrame으로 읽기 # 
import pandas as pd
import numpy as np
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('C:/RecoSys/Data/u.user', sep='|', names=u_cols, encoding='latin-1') #주어진 데이터가 latin1 이라는 방식으로 인코딩 되있어서 글자가 꺠지지 않게 불러옴.
users.head()


,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [17]:
# u.item 파일을 DataFrame으로 읽기
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('C:/RecoSys/Data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()


,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [2]:
# u.data 파일을 DataFrame으로 읽기  영화평점데이터 
import pandas as pd
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('C:/RecoSys/Data/u.data', sep='\t', names=r_cols, encoding='latin-1') 
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


In [21]:
# Best-seller 추천 
def recom_movie1(n_items):
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items] #영화의 평균평점을 내림차순으로 정렬하고 n_item만큼 movie_sort에 저장함.
    recom_movies = movies.loc[movie_sort.index] # 선정된 5개의 영화를 recom_movies에 저장하겠다.
    recommendations = recom_movies['title'] # 성전된 5개 영화의 제목을 골라서 reccommendations에 저장한 후에 이를 반환한다.
    return recommendations

movie_mean = ratings.groupby(['movie_id'])['rating'].mean() #groupby를 이용해서 영화별 평점을 구한 후에  movie_mean에 저장한다.
recom_movie1(5)

movie_id
1293                                      Star Kid (1997)
1467                 Saint of Fort Washington, The (1993)
1653    Entertaining Angels: The Dorothy Day Story (1996)
814                         Great Day in Harlem, A (1994)
1122                       They Made Me a Criminal (1939)
Name: title, dtype: object

In [27]:
from sklearn.metrics import mean_squared_error
import numpy as np

# 정확도 계산 

def RMSE(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse = []     #결과를 저장할 rmse변수를 초기화 한다.

for user in set(ratings.index):                 #ratings에 있는 user_id를 다 추출해서 한 user씩 루프를 돌린다.
    y_true = ratings.loc[user]['rating']        #각 사용자가 평가한 모든 영화의 평점을 y_true에 저장한다.
    y_pred = movie_mean[ratings.loc[user]['movie_id']] # 해당 사용자가 평가한 영화의 평점평균을 y_pred에 저장한다. 
    accuracy = RMSE(y_true, y_pred)             #실제값 y_true와 예측값 y_pred로 rmse를 구한다. (사용자 한명의 rmse)
    rmse.append(accuracy)  #현재 사용자의 rmse를 rmse 리스트에 추가한다.
print(np.mean(rmse)) # 현재 사용자의 rmse의 평균을 보여준다.

0.996007224010567
